## Epipolar geometry
set up two cameras, both with the internal parameters,
$$
K = \begin{bmatrix}
1000 & 0 & 300 \\ 0 & 1000 & 200 \\ 0 & 0 & 1
\end{bmatrix}
$$
Now, for the first camera — let us call that Cam1 — set the rotation to identity $R1 = I$ and set the translation to zero $t1 = 0$. For the second camera $Cam2$ use the rotation given by the $R$ function
$$
R_2 = R(0.7,-0.5,0.8) \\
R(\theta_x, \theta_y, \theta_z) = \begin{bmatrix}
cos(\theta_z) & -sin(\theta_z) & 0 \\ sin(\theta_z) & cos(\theta_z) & 0 \\ 0 & 0 & 1
\end{bmatrix}  \begin{bmatrix}
cos(\theta_y) & 0 & sin(\theta_y) \\ 0 & 0 & 1 \\ -sin(\theta_y) & 0 & cos(\theta_y)
\end{bmatrix} \begin{bmatrix}
1 & 0 & 0\\ 0 & cos(\theta_x) & -sin(\theta_x) \\ 0 & sin(\theta_x) & cos(\theta_x)
\end{bmatrix}
$$
and the translation
$$
t_2 = \begin{bmatrix} 0.2 \\ 2 \\ 1 \end{bmatrix}.
$$

The rotation can be constructed in Python using Rotation module from scipy as follows:
```
from scipy.spatial.transform import Rotation
R2 = Rotation.from_euler('xyz', [0.7, -0.5, 0.8]).as_matrix()
```

### Exercise 3.1
Consider the 3D point
$$
Q = \begin{bmatrix} 1 \\ 0.5 \\ 4 \\ 1 \end{bmatrix}
$$
and find the projections in $Cam1$ and $Cam2$, respectively, points $q1$ and $q2$.

In [15]:
import numpy as np
from scipy.spatial.transform import Rotation

# converts from homogeneous to inhomogeneous coordinates
def Pi(ph):
    return ph[:-1]/ph[-1]
    
# converts from inhomogeneous to homogeneous coordinates
def PiInv(p):
    ones = np.ones((1, p.shape[1]))
    ph = np.vstack((p, ones))
    return ph

def projectpoints(K, R, t, Q):
    Q_homogeneous = PiInv(Q)
    q = np.hstack((R,t)) @ Q_homogeneous
    P = Pi(q)
    p = K @ PiInv(P)
    return p

K = np.array([[1000,0,300],[0,1000,200],[0,0,1]])
R1 = np.eye(3)
R2 = Rotation.from_euler('xyz', [0.7, -0.5, 0.8]).as_matrix()
t1 = np.zeros((3,1))
t2 = np.array([[0.2,2,1]]).T
Q = np.array([[1,0.5,4,1]]).T

q1 = projectpoints(K, R1, t1, Pi(Q))
q2 = projectpoints(K, R2, t2, Pi(Q))

print('q1 = \n', Pi(q1))
print('q2 = \n', Pi(q2).round(3))

q1 = 
 [[550.]
 [325.]]
q2 = 
 [[582.473]
 [185.99 ]]


### Exercise 3.2
Implement a function CrossOp that takes a vector in 3D and returns the 3×3 matrix corresponding to taking the cross product with that vector. In the case that $p = \begin{bmatrix} x & y & z \end{bmatrix}^T $ you should have
$$
CrossOp = [p]_\times = \begin{bmatrix} 0 & -z & y \\ z & 0 & -x \\ -y & 0 & x \end{bmatrix}
$$
As a good habit, verify that your function works by testing it on random vectors to ensure that
$$
[p_1]_\times p_2 = p_1 \times p_2
$$

In [26]:
def CrossOp(p):
    return np.array([[0, -p[2], p[1]],
                     [p[2], 0, -p[0]],
                     [-p[1], p[0], 0]])

p1 = np.random.rand(3)
p2 = np.random.rand(3)
print("[p1]xp2 = ", CrossOp(p1) @ p2)
print("p1 x p2 = ", np.cross(p1,p2))

[p1]xp2 =  [-0.11579164  0.2049554  -0.03060905]
p1 x p2 =  [-0.11579164  0.2049554  -0.03060905]
